In [9]:
import numpy as np
import itertools
from scipy.linalg import toeplitz
# import os
# import json
import sys
# import glob
# from scipy import stats
# from scipy.signal import argrelextrema
# from sklearn import linear_model
# import datetime
# import cv2
# import time
# import h5py
# from itertools import groupby
# from math import ceil
import pickle
# import math
# from scipy.linalg import norm


# %qtconsole
%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt
# from matplotlib import cm
# from matplotlib import colors
# from matplotlib.patches import Rectangle, Circle
# from matplotlib import patches
# from matplotlib.collections import PatchCollection
# import matplotlib.gridspec as gridspec
# from matplotlib.colors import LinearSegmentedColormap
# from mpl_toolkits.mplot3d import Axes3D

# import statsmodels.api as sm

sys.path.append('/home/gravishlab/Documents/Python/')
sys.path.append('/home/gravishlab/Documents/Python/AntTrackCode')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/')
sys.path.append('/home/gravishlab/Documents/Python/Tracker/Tracker/')
from Tracker.Tracker import Tracker

# import multiprocessing
# import threading
# from subprocess import call

import pandas as pd
# import seaborn as sns

# import copy

# stop telling me i have a nan in an array with a logical comparison
with np.errstate(invalid='ignore'):
    np.less([np.nan, 0], 1)

Using matplotlib backend: Qt5Agg


In [19]:
%qtconsole

In [56]:
# load in neutral leg locations
vid_locations = '/media/gravishlab/SeagateExpansionDrive/AntTrack/'
with open('%sAnt_Straight_Foot_Placement.pkl'%vid_locations, 'rb') as f:
    footX, footY = pickle.load(f)
    
# TUNNEL EXPERIMENTS
pix2mm = 959.7563/30 # Measured 3cm in 4 cameras
fps = 239.16
v_multiplier= pix2mm * fps
print('%0.3f pixels per mm'%pix2mm)

pltcolors = ['#464F56', '#BA4246', '#087E8B', '#701C6F']

31.992 pixels per mm


## MODEL where ant limbs can touchdown based on body location

In [126]:
# USER INPUT
data = []
p_val = 50 # how many division for each box

plt.close('all')

for box_size in [1,3,5]:
    
    precision = box_size/p_val
    plot_things = False
    print('box_size is %i, precision %0.2f'%(box_size, precision))

    floor_x_range = [-5,10]
    floor_y_range = [-5,10]
    h_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
    v_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
    heights = toeplitz(np.arange(0,len(h_lines))%2,r=np.arange(0,len(v_lines))%2)-1
    colors = ['grey','silver']

    # neutral conditions
    neutral_body_height = 2/3
    flatX=np.mean(np.reshape(footX[1,0,:],[2,3]).T,axis=1)/pix2mm # ave across R and L to get symmetrical --> hind, mid, fore
    flatY=np.mean(np.abs(np.reshape(footY[1,0,:],[2,3])).T,axis=1)/pix2mm * [1,-1,1]
    leg_lengths = np.linalg.norm(np.array([flatX,flatY,neutral_body_height*np.ones(3)]),axis=0) # leg lengths from stance
    leg_lengths = leg_lengths*1.1

    # for a given body location
    for body_height in neutral_body_height+np.linspace(0,-1,10):

        # initialize figure
        if plot_things:
            plt.figure()
            plt.title('body height = %0.2f mm'%body_height)
            for hh,h in enumerate(h_lines[:-1]):
                for vv,v in enumerate(v_lines[:-1]):
                    rect = patches.Rectangle((h,v), box_size, box_size, color = colors[heights[hh,vv]+1])
                    plt.gca().add_patch(rect)
                    plt.show()
            plt.xlim(floor_x_range)
            plt.xlim(floor_y_range)
            plt.axis('equal')

        for body_y in np.arange(0,box_size+precision, precision):

            for body_x in np.arange(0,2*box_size+precision, precision):
                foot_xs, foot_ys, foot_thetas, foot_phis, foot_dists, foot_heights = np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan), np.full((2,3),np.nan)
                body_loc = [body_x,body_y]

                if plot_things:
                    plt.plot(body_x,body_y,'+k')
                    foot_colors = ['r','g','b']
                    line_styles = ['--','-']

                # for each foot, 
                for ff in [0,1,2]:
                    foot_x = body_loc[0]+flatX[ff]
                    foot_y = body_loc[1]+flatY[ff]
                    angOI = np.arctan2(flatY[ff],flatX[ff])

                    if plot_things:
                        plt.plot(foot_x,foot_y, '+', color = foot_colors[ff], alpha = 0.5)

                    for zz in [-1,0]: 

                        if body_height-zz>leg_lengths[ff]: # if leg can't touch surface
                            continue
                            
                        if body_height<(np.floor(body_y/box_size)-1): # body height is lower than would actually be able to go on checkerboard
                            continue

                        # find best touchdown location on that surface
                        r= (leg_lengths[ff]**2-(zz-body_height)**2)**(1/2) 
                        thetas = np.arange(-180,180,0.1)
                        arc_xs = r*np.cos(np.deg2rad(thetas))+body_x
                        arc_ys = r*np.sin(np.deg2rad(thetas))+body_y
                        arc_rows = np.argmax(np.repeat(arc_ys[np.newaxis,:],len(v_lines),axis=0)<np.repeat(v_lines[:,np.newaxis],len(thetas),axis = 1), axis =0)-1
                        arc_cols = np.argmax(np.repeat(arc_xs[np.newaxis,:],len(h_lines),axis=0)<np.repeat(h_lines[:,np.newaxis],len(thetas),axis = 1), axis =0)-1
                        arc_heights = heights[arc_rows,arc_cols]
                        good_thetas = (arc_heights ==zz) & (np.abs(thetas-np.rad2deg(angOI))<30) & (np.sign(thetas)==np.sign(angOI))
                        arc_dist_to_neutral = np.linalg.norm(np.vstack([arc_xs-foot_x,arc_ys-foot_y]),axis =0)
                        if np.sum(good_thetas)==0:
                            continue
                        theta_min = np.min(thetas[good_thetas])
                        theta_max = np.max(thetas[good_thetas])
                        tmp = arc_dist_to_neutral.copy().astype(float)
                        tmp[np.logical_not(good_thetas)]=np.nan
                        best_theta_idc = np.nanargmin(tmp)
                        del tmp

                        if plot_things:
                            h_arc = patches.Arc(body_loc, 2*r, 2*r, angle = 0, theta1=theta_min, theta2=theta_max , color = foot_colors[ff], 
                                            linestyle = line_styles[zz+1], alpha = 0.8)
                            plt.gca().add_patch(h_arc)
                            plt.plot(arc_xs[best_theta_idc], arc_ys[best_theta_idc], '.', color = foot_colors[ff], alpha = 0.5)
                            plt.plot()
                            plt.show()

                        # save to variables
                        foot_xs[zz+1,ff]=arc_xs[best_theta_idc]
                        foot_ys[zz+1,ff]=arc_ys[best_theta_idc]
                        foot_thetas[zz+1,ff]=thetas[best_theta_idc]
                        foot_phis[zz+1,ff]=np.rad2deg(np.arccos(r/leg_lengths[ff]))
                        foot_dists[zz+1,ff]=arc_dist_to_neutral[best_theta_idc]
                        foot_heights[zz+1,ff]=arc_heights[best_theta_idc]

                # append data for each valid stance
                tmp =foot_xs.copy()
                tmp[np.isfinite(tmp)]=np.sum(np.argwhere(np.isfinite(foot_xs))*np.array([3,1]),axis=1)
                list_of_TDs = [i[np.isfinite(i)].astype(int).tolist() for i in np.split(tmp.T,3, axis =0)]
                if len(list_of_TDs)<3:
                    continue
                stance_idcs = list(itertools.product(*list_of_TDs))
                for ss in range(len(stance_idcs)):
                    dict_to_df = {}
                    dict_to_df['box_size']=box_size
                    dict_to_df['body_x']=body_x
                    dict_to_df['body_y']=body_y
                    dict_to_df['body_z']=body_height
                    dict_to_df['foot_xs']=foot_xs.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_ys']=foot_ys.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_d_xs']=foot_xs.flatten()[np.array(stance_idcs[ss])] - (body_loc[0]+flatX)
                    dict_to_df['foot_d_ys']=foot_ys.flatten()[np.array(stance_idcs[ss])] - (body_loc[1]+flatY)
                    dict_to_df['foot_thetas']=foot_thetas.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_phis']=foot_phis.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_dists']=foot_dists.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['foot_heights']=foot_heights.flatten()[np.array(stance_idcs[ss])]
                    dict_to_df['stance_type']=['flat','pitch','roll'][int(np.sum(np.abs(np.diff( foot_heights.flatten()[np.array(stance_idcs[ss])]))))]
                    dict_to_df['stance_displacement']=np.sum(foot_dists.flatten()[np.array(stance_idcs[ss])])
                    dict_to_df['stance_height']=['valley','peak'][(np.sum(foot_heights.flatten()[np.array(stance_idcs[ss])])>=-1).astype(int)]

                    # append to dict
                    data.append(dict_to_df.copy())


foot_placement_df = pd.DataFrame(data)
print('\ndone making foot placement dataframe')
print(len(foot_placement_df))

box_size is 1, precision 0.02
box_size is 3, precision 0.06
box_size is 5, precision 0.10

done making foot placement dataframe
715670


In [185]:
# analyze dataframe of foot placement info
plt.close('all')

plot_heatmaps = False

floor_x_range = [-5,10]
floor_y_range = [-5,10]
h_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
v_lines = np.hstack([np.flip(-1*np.arange(0,5,box_size)[1:], axis =0),np.arange(0,10,box_size)])
heights = toeplitz(np.arange(0,len(h_lines))%2,r=np.arange(0,len(v_lines))%2)-1
colors = ['grey','silver']
text_locations = np.meshgrid(h_lines[2:4]-box_size/2,v_lines[1::2]-box_size/2)



# 2d histogram
n_stances = np.full((2,3,3), np.nan)
n_stances_adjusted = np.full((2,3,3), np.nan)
stance_types = ['flat','pitch','roll']
for bb,box_size in enumerate([1,3,5]):
    # where are box cut-offs for 2d histogram
    xedges = np.arange(floor_x_range[0],floor_x_range[1]+3*box_size/p_val,box_size/p_val*2)-box_size/p_val
    yedges = np.arange(floor_y_range[0],floor_y_range[1]+3*box_size/p_val,box_size/p_val*2)-box_size/p_val
    if plot_heatmaps:
        fig=plt.figure(figsize = (9,5))
    
    for ss,s_type in enumerate(stance_types):
        for hh,h_type in enumerate(['valley', 'peak']):
            df_OI = foot_placement_df.loc[(foot_placement_df['box_size']==box_size) & (foot_placement_df['stance_type']==s_type) & (foot_placement_df['stance_height']==h_type) ]
            n_stances[hh,ss,bb] = len(df_OI)
            n_stances_adjusted[hh,ss,bb] = np.sum(df_OI['foot_dists'].apply(lambda x: 1/np.sum(x)).values)
            print('stance type: %s, height: %s, n: %i, adjusted: %0.1f'%(s_type, h_type, n_stances[hh,ss,bb], n_stances_adjusted[hh,ss,bb]))

            H, _,_ = np.histogram2d(df_OI['body_x'].values, df_OI['body_y'].values, bins = (xedges,yedges)) # count number of stances
        #     H, _,_ = np.histogram2d(df_OI['body_x'].values, df_OI['body_y'].values, bins = (xedges,yedges) # weight each stance by "cost" (how much cumulative distance from neutral position)
        #                             , weights = 1/foot_placement_df.loc[(foot_placement_df['stance_type']==s_type)]['stance_displacement'].values)
            H = H.T
            
            if plot_heatmaps:
                plt.subplot(2,3,ss+1+hh*3)
                for ii in range(0,3):
                    plt.axvline(x=box_size*ii, color ='w', linestyle='-')
                    plt.axhline(y=box_size*ii, color ='w', linestyle='-')
                for jj in range(1,3):
#                     plt.axvline(x=box_size*jj-(box_size/2)-0.5, color ='r', linestyle=':')
                    plt.plot(np.ones(2)*box_size*jj-(box_size/2)-0.5,[0,box_size], color ='r', linestyle=':')
                    plt.plot(np.ones(2)*box_size*jj-(box_size/2)+0.5,[0,box_size], color ='r', linestyle=':')
#                     plt.axvline(x=box_size+jj-(box_size/2)+0.5, color ='r', linestyle=':')
                X,Y = np.meshgrid(xedges, yedges)
                pc=plt.gca().pcolormesh(X,Y,H, vmin = 0, vmax = 100) # number stanced
            #     pc=plt.gca().pcolormesh(X,Y,H, vmin = 0, vmax = 80) # weighted
                plt.axis('equal')
                plt.xlim([-1*box_size,3*box_size])
                plt.ylim([-1*box_size,box_size*2])
                if hh == 0:
                    plt.title('%s stances'%s_type)
                    plt.gca().get_xaxis().set_visible(False)
                for ttx, tx in enumerate(range(0,box_size*6,box_size)):
                    for tty, ty in enumerate(range(0,box_size*3,box_size)):
                        if (tty==1) & ((ttx==1) or (ttx==2)):
                            continue
                        plt.text(tx-box_size/2,ty-box_size/2, ['v','p'][(ttx+tty)%2], color ='w' , ha='center')
    if plot_heatmaps:
        cax = fig.add_axes([0.92,0.1,0.02,0.8])
        #         plt.colorbar(pc, cax = cax, label = 'more <---> less displacement')
        plt.colorbar(pc, cax = cax, label = '# stances')





# plot stacked bar graph of number of stances
# plt.figure()
# bcolors = ['r','g','b']
# for bb,box_size in enumerate([1,3,5]):
#     total_n = np.sum(np.sum(n_stances[:,:,bb]))
#     total_n_adjusted = np.sum(np.sum(n_stances_adjusted[:,:,bb]))
#     for ss in range(0,3):
#         for hh in range(0,2):
#             plt.bar(box_size-1, n_stances[hh,ss,bb]/total_n, width = 1/2, 
#                     bottom = np.insert(np.cumsum(n_stances[:,:,bb].T.flatten()),0,0)[ss*2+hh]/total_n,  color = bcolors[ss], alpha = 0.2+0.3*hh)
#             plt.bar(box_size-1/3, n_stances_adjusted[hh,ss,bb]/total_n_adjusted, width = 1/2, 
#                     bottom = np.insert(np.cumsum(n_stances_adjusted[:,:,bb].T.flatten()),0,0)[ss*2+hh]/total_n_adjusted,  color = bcolors[ss], alpha = 0.2+0.3*hh)
#     plt.text(-9.5,0.95-0.1*bb,'total n stances: %i'%total_n)
    
# plt.xlim([-10,6])
# plt.gca().get_xaxis().set_ticks([])
# plt.xlabel('1, 3, 5 mm substrates (regular/adjusted)')
# plt.ylabel('proportion of stances')




# # plt point clouds of where feet place down for model
# plt.figure(figsize = (12,10))
# for ss,s_type in enumerate(stance_types):
    
#         df_OI = foot_placement_df.loc[(foot_placement_df['stance_type']==s_type)]
#         d_x = np.vstack(df_OI['foot_d_xs'].values)
#         d_y = np.vstack(df_OI['foot_d_ys'].values)
#         heights = np.vstack(df_OI['foot_heights'].values)
        
#         for ll in range(0,3):
#             plt.subplot(3,3, ss*3+ll+1)
# #             plt.xlim([-1.25,1.25])
            
#             for hh in [-1,0]:
#                 where_height = heights[:,ll]==hh
#                 plt.plot(d_x[where_height,ll], d_y[where_height,ll]*(ll%2*2-1), '.', color = ['b','g'][hh+1], alpha = 0.02, MarkerSize = 2)
#                 plt.plot(np.mean(d_x[where_height,ll]), np.mean(d_y[where_height,ll]*(ll%2*2-1)), '+', color = ['b','g'][hh+1])
#                 plt.plot([0,np.mean(d_x[where_height,ll])], [0,np.mean(d_y[where_height,ll])*(ll%2*2-1)], '-', color = 'k')
                
# #                 weights_to_use = np.vstack(1/df_OI['foot_dists'].values)[where_height, ll]
#                 weights_to_use = 1/df_OI['stance_displacement'].values[where_height]
#                 x_mean = np.average(d_x[where_height,ll], weights = weights_to_use)
#                 y_mean = np.average(d_y[where_height,ll]*(ll%2*2-1),  weights = weights_to_use)
#                 plt.plot([0,x_mean], [0,y_mean], '--', color = 'r')
#                 plt.plot(0,0,'.k')

#             if ll==0:
#                 plt.ylabel('%s'%s_type)
#             else:
#                 plt.gca().get_yaxis().set_visible(False)
#             if ss ==0:
#                 plt.title('%s'%['fore','mid','hind'][ll])
#             plt.ylim([-1.25,1.25])
#             plt.xlim([-1.25,1.25])
            
            
# plot distribution of stances along y-axis for 1mm range in center of block
plt.figure()
for bb,box_size in enumerate([1,3,5]):
    
    vals_OI = np.abs(foot_placement_df.loc[(foot_placement_df['box_size']==box_size)]['body_x'].values%box_size-box_size/2)<=0.5
    stance_ys_OI = np.abs( foot_placement_df.loc[(foot_placement_df['box_size']==box_size)]['body_y'].values[vals_OI]/box_size - 0.5)/0.5
    # only flat values
#     vals_OI = np.abs(foot_placement_df.loc[(foot_placement_df['box_size']==box_size) & (foot_placement_df['stance_type']=='flat')]['body_x'].values%box_size-box_size/2)<=0.5
#     stance_ys_OI = np.abs( foot_placement_df.loc[(foot_placement_df['box_size']==box_size) & (foot_placement_df['stance_type']=='flat')]['body_y'].values[vals_OI]/box_size - 0.5)/0.5

    hprecision = 50/2 # how many bins per 1 unit
    hbins = np.linspace(0, 1, hprecision+1)-0.01

    # no weighting
#     h_counts = np.histogram(stance_ys_OI, bins = hbins )[0]
    
    # weight by stance displacement
#     h_counts = np.histogram(stance_ys_OI, bins = hbins, weights = foot_placement_df.loc[(foot_placement_df['box_size']==box_size)]['stance_displacement'].values[vals_OI])[0] # weighted by stance displacement

    # one type of stance weighted more
    s_type_OI = 'pitch' #'flat'
    weighting = 10
    h_counts = np.histogram(stance_ys_OI, bins = hbins, weights = (foot_placement_df.loc[(foot_placement_df['box_size']==box_size)]['stance_type'].values[vals_OI]==s_type_OI)*weighting+1)[0] # flat weighted more
    plt.title('%s stances weighted %i times more'%(s_type_OI, weighting))

    h_counts[0] = 2*h_counts[0]
    n_data = np.sum(stance_ys_OI)
    h_counts = h_counts/n_data
    plt.plot(hbins[:-1][h_counts>0]+(1/(2*hprecision)), h_counts[h_counts>0], ':', color = pltcolors[bb+1])
    plt.bar(hbins[:-1][h_counts>0], h_counts[h_counts>0], color = pltcolors[bb+1], alpha = 0.2, width =1/hprecision, align='edge')
    # plt.hist(stance_ys_OI, bins = hbins, color = pltcolors[2], alpha = 0.2, weights = np.ones(stance_ys_OI.shape)*(1/n_data))
    plt.xlabel('center   <------>   edges')
    plt.ylabel('proportion of stances')
    
#         # lowpass filter histogram data
#         b,a = signal.butter(2, 1/(precision/4))
#         temp = signal.filtfilt(b,a, np.hstack([np.flipud(h_counts[h_counts>0]),h_counts[h_counts>0], np.flipud(h_counts[h_counts>0])]) , padlen = 10)
#         h_counts_filt= temp[ len(h_counts[h_counts>0]):(2*len(h_counts[h_counts>0]))]
#         plt.plot(bins[:-1][h_counts>0]+(1/(2*precision)), h_counts_filt, linestyle = line_styles[height], color = pltcolors[ss])
#         plt.text(0.72+height/10, .06-ss*0.003, 'n: %i'%len(hist_OI[heights==height]), color  = pltcolors[ss])
#         plt.ylim([0,0.07])



# del df_OI

stance type: flat, height: valley, n: 47734, adjusted: 78806.7
stance type: flat, height: peak, n: 51183, adjusted: 55021.0
stance type: pitch, height: valley, n: 96084, adjusted: 117405.6
stance type: pitch, height: peak, n: 101210, adjusted: 112718.7
stance type: roll, height: valley, n: 50034, adjusted: 63535.8
stance type: roll, height: peak, n: 48351, adjusted: 50216.1
stance type: flat, height: valley, n: 18670, adjusted: 28288.8
stance type: flat, height: peak, n: 24115, adjusted: 23473.2
stance type: pitch, height: valley, n: 52482, adjusted: 66787.9
stance type: pitch, height: peak, n: 59147, adjusted: 65760.1
stance type: roll, height: valley, n: 22375, adjusted: 25041.1
stance type: roll, height: peak, n: 21946, adjusted: 20181.9
stance type: flat, height: valley, n: 16358, adjusted: 32338.6
stance type: flat, height: peak, n: 18946, adjusted: 20490.1
stance type: pitch, height: valley, n: 29710, adjusted: 38637.8
stance type: pitch, height: peak, n: 30290, adjusted: 33627.4

/home/gravishlab/.virtualenvs/ants/lib/python3.6/site-packages/ipykernel_launcher.py:141: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


In [211]:
# Keep one stance per x,y,z location

plot_heatmaps = True
plt.close('all')
n_stances = np.full((2,3,3), np.nan)
n_stances_adjusted = np.full((2,3,3), np.nan)
stance_types = ['flat','pitch','roll']
for bb,box_size in enumerate([1,3,5]):
    # where are box cut-offs for 2d histogram
    xedges = np.arange(floor_x_range[0],floor_x_range[1]+3*box_size/p_val,box_size/p_val*2)-box_size/p_val
    yedges = np.arange(floor_y_range[0],floor_y_range[1]+3*box_size/p_val,box_size/p_val*2)-box_size/p_val
    fig = plt.figure(figsize=(12,7))
    
    
    tdf_OI = foot_placement_df.loc[(foot_placement_df['box_size']==box_size)][['body_x','body_y', 'body_z','stance_type','stance_height','stance_displacement']]#['body_x':'stance_displacement']
    min_displacement_idcs = tdf_OI.groupby(['body_x','body_y','body_z'],as_index=False).idxmin()['stance_displacement'].values
    del tdf_OI
    
    
    for ss,s_type in enumerate(stance_types):
        for hh,h_type in enumerate(['valley', 'peak']):
            df_OI = foot_placement_df.iloc[min_displacement_idcs].loc[(foot_placement_df['stance_type']==s_type) & (foot_placement_df['stance_height']==h_type) ]
            n_stances[hh,ss,bb] = len(df_OI)
            n_stances_adjusted[hh,ss,bb] = np.sum(df_OI['foot_dists'].apply(lambda x: 1/np.sum(x)).values)
            print('stance type: %s, height: %s, n: %i, adjusted: %0.1f'%(s_type, h_type, n_stances[hh,ss,bb], n_stances_adjusted[hh,ss,bb]))

            H, _,_ = np.histogram2d(df_OI['body_x'].values, df_OI['body_y'].values, bins = (xedges,yedges)) # count number of stances
        #     H, _,_ = np.histogram2d(df_OI['body_x'].values, df_OI['body_y'].values, bins = (xedges,yedges) # weight each stance by "cost" (how much cumulative distance from neutral position)
        #                             , weights = 1/foot_placement_df.loc[(foot_placement_df['stance_type']==s_type)]['stance_displacement'].values)
            H = H.T
            
            if plot_heatmaps:
                plt.subplot(2,3,ss+1+hh*3)
                for ii in range(0,3):
                    plt.axvline(x=box_size*ii, color ='w', linestyle='-')
                    plt.axhline(y=box_size*ii, color ='w', linestyle='-')
                for jj in range(1,3):
#                     plt.axvline(x=box_size*jj-(box_size/2)-0.5, color ='r', linestyle=':')
                    plt.plot(np.ones(2)*box_size*jj-(box_size/2)-0.5,[0,box_size], color ='r', linestyle=':')
                    plt.plot(np.ones(2)*box_size*jj-(box_size/2)+0.5,[0,box_size], color ='r', linestyle=':')
#                     plt.axvline(x=box_size+jj-(box_size/2)+0.5, color ='r', linestyle=':')
                X,Y = np.meshgrid(xedges, yedges)
                pc=plt.gca().pcolormesh(X,Y,H, vmin = 0, vmax = 50) # number stanced
            #     pc=plt.gca().pcolormesh(X,Y,H, vmin = 0, vmax = 80) # weighted
                plt.axis('equal')
                plt.xlim([-1*box_size,3*box_size])
                plt.ylim([-1*box_size,box_size*2])
                if hh == 0:
                    plt.title('%s stances'%s_type)
                    plt.gca().get_xaxis().set_visible(False)
                for ttx, tx in enumerate(range(0,box_size*6,box_size)):
                    for tty, ty in enumerate(range(0,box_size*3,box_size)):
                        if (tty==1) & ((ttx==1) or (ttx==2)):
                            continue
                        plt.text(tx-box_size/2,ty-box_size/2, ['v','p'][(ttx+tty)%2], color ='w' , ha='center')
    if plot_heatmaps:
        cax = fig.add_axes([0.92,0.1,0.02,0.8])
        #         plt.colorbar(pc, cax = cax, label = 'more <---> less displacement')
        plt.colorbar(pc, cax = cax, label = '# stances')
        
        
        
# plot stacked bar graph of number of stances
plt.figure()
bcolors = ['r','g','b']
for bb,box_size in enumerate([1,3,5]):
    total_n = np.sum(np.sum(n_stances[:,:,bb]))
    total_n_adjusted = np.sum(np.sum(n_stances_adjusted[:,:,bb]))
    for ss in range(0,3):
        for hh in range(0,2):
            plt.bar(box_size-1, n_stances[hh,ss,bb]/total_n, width = 1/2, 
                    bottom = np.insert(np.cumsum(n_stances[:,:,bb].T.flatten()),0,0)[ss*2+hh]/total_n,  color = bcolors[ss], alpha = 0.2+0.3*hh)
            plt.bar(box_size-1/3, n_stances_adjusted[hh,ss,bb]/total_n_adjusted, width = 1/2, 
                    bottom = np.insert(np.cumsum(n_stances_adjusted[:,:,bb].T.flatten()),0,0)[ss*2+hh]/total_n_adjusted,  color = bcolors[ss], alpha = 0.2+0.3*hh)
    plt.text(-9.5,0.95-0.1*bb,'total n stances: %i'%total_n)
    
plt.xlim([-10,6])
plt.gca().get_xaxis().set_ticks([])
plt.xlabel('1, 3, 5 mm substrates (regular/adjusted)')
plt.ylabel('proportion of stances')

stance type: flat, height: valley, n: 13984, adjusted: 41386.8
stance type: flat, height: peak, n: 13333, adjusted: 17229.6
stance type: pitch, height: valley, n: 11212, adjusted: 20356.7
stance type: pitch, height: peak, n: 5110, adjusted: 6079.4
stance type: roll, height: valley, n: 7362, adjusted: 12371.5
stance type: roll, height: peak, n: 205, adjusted: 199.7
stance type: flat, height: valley, n: 4543, adjusted: 14411.5
stance type: flat, height: peak, n: 4312, adjusted: 5557.0
stance type: pitch, height: valley, n: 16778, adjusted: 30270.9
stance type: pitch, height: peak, n: 19021, adjusted: 23422.1
stance type: roll, height: valley, n: 3764, adjusted: 6582.3
stance type: roll, height: peak, n: 1703, adjusted: 1255.0
stance type: flat, height: valley, n: 6787, adjusted: 23618.5
stance type: flat, height: peak, n: 10499, adjusted: 13101.7
stance type: pitch, height: valley, n: 10582, adjusted: 17688.9
stance type: pitch, height: peak, n: 10946, adjusted: 13540.4
stance type: roll

Text(0,0.5,'proportion of stances')